# Import

In [14]:
import numpy as np
import pandas as pd
import datetime

import sys, os
sys.path.insert(0, os.path.abspath('scripts'))
import rus_table_parser

# Data parsing

In [15]:
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 5
# MONTH = 6

# Save of "https://coronavirus-monitor.ru/coronavirus-v-rossii/" page
path = f'html_data/{DAY}-{MONTH}/Коронавирус в России. Онлайн карта распространения коронавируса в России..html'
parse_df = rus_table_parser.run_parsing(path)
parse_df

,Confirmed,Date,Deaths,Recovered,Region/City
0,197018.0,2020-06-08,2970.0,104347.0,Москва
1,44983.0,2020-06-08,578.0,12031.0,Московская область
2,18835.0,2020-06-08,392.0,7915.0,Санкт-Петербург
3,12442.0,2020-06-08,140.0,4878.0,Нижегородская область
4,7668.0,2020-06-08,43.0,3273.0,Свердловская область
...,...,...,...,...,...
80,204.0,2020-06-08,0.0,71.0,Сахалинская область
81,180.0,2020-06-08,0.0,101.0,Республика Алтай
82,177.0,2020-06-08,3.0,145.0,Севастополь
83,106.0,2020-06-08,1.0,65.0,Чукотский автономный округ


# Preparing parsed data

In [16]:
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
5505,2020-06-07,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,115.0,0.0,14.0,3346.0,20.0,1126.0
5506,2020-06-07,Чеченская Республика,Chechen Republic,95.0,19.0,0.0,2.0,1345.0,14.0,990.0
5507,2020-06-07,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,4.0,0.0,2.0,106.0,1.0,65.0
5508,2020-06-07,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,73.0,0.0,53.0,2694.0,11.0,1429.0
5509,2020-06-07,Ярославская область,Yaroslavl region,76.0,70.0,0.0,7.0,3646.0,15.0,1176.0


In [17]:
# Create day-columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

In [18]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b')
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

In [19]:
# Rename regions
rename_dict = {
    'Татарстан' : 'Республика Татарстан',
    'Башкортостан' : 'Республика Башкортостан',
    'Чувашская Республика': 'Республика Чувашия',
    'Камчатский край' : 'Камчатский край',
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
    'Республика Алтай' : 'Республика Алтай'
}

parse_df['Region/City'] = parse_df['Region/City'].replace(rename_dict)

In [20]:
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# Minus a day from now if needed
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [21]:
# Fit day-columns
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Confirmed'].max()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Deaths'].max()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Recovered'].max()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [22]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [23]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 476768.0 Day-Confirmed: 8990.0
Russia Deaths: 5997.0 Day-Deaths: 114.0
Russia Recovered: 230938.0 Day-Recovered: 4044.0


# Data saving

In [24]:
# Save full snapshot of current day
parse_df.to_csv(f'covid19-russia-cases-upd-full.csv', index=False)

In [25]:
# Filter regions without changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [26]:
# Save parsed data for future use
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd.csv', index=False)